In [ ]:
!pip install konlpy
!pip install pymysql
!pip install emoji


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 65.5 MB/s 
     |████████████████████████████████| 453 kB 69.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 225 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 175 kB 10.5 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=f37551fded54f004babdc3f903a9f84e0f1e46113b70e7c42fcacde8f06a89df
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji


In [ ]:
import konlpy
import tensorflow as tf
import pandas as pd
import urllib.request
import matplotlib.pyplot as plt
import re
from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pymysql
import emoji
from sklearn.model_selection import train_test_split
import pickle
from tensorflow.keras.layers import Embedding, Dense, LSTM , Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
def extract_emojis(text):
    for c in text:
        if c in emoji.UNICODE_EMOJI['en']:
          text = re.sub(c, "", text)
    
    return text

def below_threshold_len(max_len, nested_list):
  cnt = 0
  for s in nested_list:
    if(len(s) <= max_len):
        cnt = cnt + 1
  print('전체 샘플 중 길이가 {} 이하인 샘플의 비율: {}'.format(max_len, (cnt / len(nested_list))*100))

def mean_word_predict_morpheme(new_sentence):
  new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
  encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
  pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
  score = float(loaded_model_morpheme.predict(pad_new)) # 예측
  if(score > 0.5):
    print("{:.2f}% 확률로 욕설입니다.\n".format(score * 100))
  else:
    print("{:.2f}% 확률로 욕설이 아닙니다.\n".format((1 - score) * 100))

In [ ]:
okt = Okt()
okt.morphs('이 바보야 진짜 아니야', stem=True)

['이', '바보', '야', '진짜', '아니다']

In [ ]:
conn = pymysql.connect(
    user='study',
    passwd='hellobee1234',
    host='121.134.135.223',
    port=3306,
    db='data'
)

cursor = conn.cursor(pymysql.cursors.DictCursor)
sql = "SELECT * FROM data.dataset;"
cursor.execute(sql)
result = cursor.fetchall()
data = pd.DataFrame(result)
data.dropna(inplace=True)
data['records'] = data['records'].map(extract_emojis)
data['records'] = data['records'].str.replace(r'\[\d+\]', "")
data['records'] = data['records'].str.replace("\xa0", "")
data.dropna(inplace=True)
data.drop_duplicates(subset=['records'], inplace=True)

X_train, X_test, y_train, y_test = train_test_split(data['records'], data['label'], test_size = 0.3, random_state = 123)

okt = Okt()

train_record = []
for sentence in tqdm(X_train):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    train_record.append(tokenized_sentence)

test_record = []
for sentence in tqdm(X_test):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    test_record.append(tokenized_sentence)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_record)
 
total_cnt = len(tokenizer.word_index) # 단어의 수

tokenizer = Tokenizer(total_cnt) # 희귀단어 집합 제거 안 함
tokenizer.fit_on_texts(train_record)

with open('wordlstm.pkl', 'wb') as f:
	pickle.dump(tokenizer, f, protocol=pickle.HIGHEST_PROTOCOL)

#텍스트 시퀀스를 정수 시퀀스로 변환
morpheme_X_train = tokenizer.texts_to_sequences(train_record) 
morpheme_X_test = tokenizer.texts_to_sequences(test_record)

max_len = 20
below_threshold_len(max_len, morpheme_X_train)

morpheme_X_train = pad_sequences(morpheme_X_train, maxlen = max_len)
morpheme_X_test = pad_sequences(morpheme_X_test, maxlen = max_len)

model = Sequential()
model.add(Embedding(total_cnt, 100)) 
model.add(LSTM(100, dropout=0.6))
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(patience=4,restore_best_weights=True)
mc_m = ModelCheckpoint('./model/best_model_mean_words_morpheme.h5', save_best_only=True)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(morpheme_X_train, y_train, epochs=100, callbacks=[es, mc_m], batch_size=16, validation_split=0.2)

print("\n Accuracy: %.4f" % (model.evaluate(morpheme_X_test, y_test)[1]))

loaded_model_morpheme = load_model('./model/best_model_mean_words_morpheme.h5')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  app.launch_new_instance()
100%|██████████| 17037/17037 [00:15<00:00, 1088.67it/s]


전체 샘플 중 길이가 20 이하인 샘플의 비율: 99.78867924528302
Epoch 1/100
1988/1988 [==============================] - 24s 8ms/step - loss: 0.2746 - accuracy: 0.8851 - val_loss: 0.1988 - val_accuracy: 0.9255
Epoch 2/100
1988/1988 [==============================] - 16s 8ms/step - loss: 0.1366 - accuracy: 0.9497 - val_loss: 0.2008 - val_accuracy: 0.9220
Epoch 3/100
1988/1988 [==============================] - 15s 7ms/step - loss: 0.0956 - accuracy: 0.9652 - val_loss: 0.2055 - val_accuracy: 0.9281
Epoch 4/100
1988/1988 [==============================] - 14s 7ms/step - loss: 0.0725 - accuracy: 0.9746 - val_loss: 0.2387 - val_accuracy: 0.9273
Epoch 5/100
533/533 [==============================] - 2s 3ms/step - loss: 0.2023 - accuracy: 0.9255

 Accuracy: 0.9255


In [ ]:
model = Sequential()
model.add(Embedding(total_cnt, 100)) 
model.add(LSTM(100, dropout=0.6))
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(patience=4,restore_best_weights=True)
mc_m = ModelCheckpoint('./model/best_model_mean_words_morpheme.h5', save_best_only=True)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(morpheme_X_train, y_train, epochs=100, callbacks=[es, mc_m], batch_size=16, validation_split=0.2)

print("\n Accuracy: %.4f" % (model.evaluate(morpheme_X_test, y_test)[1]))

loaded_model_morpheme = load_model('./model/best_model_mean_words_morpheme.h5')

Epoch 1/100
1988/1988 [==============================] - 19s 8ms/step - loss: 0.2733 - accuracy: 0.8858 - val_loss: 0.1935 - val_accuracy: 0.9282
Epoch 2/100
1988/1988 [==============================] - 16s 8ms/step - loss: 0.1385 - accuracy: 0.9494 - val_loss: 0.1839 - val_accuracy: 0.9316
Epoch 3/100
1988/1988 [==============================] - 14s 7ms/step - loss: 0.0965 - accuracy: 0.9653 - val_loss: 0.2018 - val_accuracy: 0.9272
Epoch 4/100
1988/1988 [==============================] - 14s 7ms/step - loss: 0.0728 - accuracy: 0.9742 - val_loss: 0.2112 - val_accuracy: 0.9230
Epoch 5/100
1988/1988 [==============================] - 14s 7ms/step - loss: 0.0581 - accuracy: 0.9800 - val_loss: 0.2369 - val_accuracy: 0.9209
Epoch 6/100
533/533 [==============================] - 2s 3ms/step - loss: 0.1917 - accuracy: 0.9322

 Accuracy: 0.9322


In [ ]:
mean_word_predict_morpheme('시발')
mean_word_predict_morpheme('시2발')
mean_word_predict_morpheme('씨2발')
mean_word_predict_morpheme('시3발')
mean_word_predict_morpheme('씨3발')
mean_word_predict_morpheme('시4발')
mean_word_predict_morpheme('씨4발')
mean_word_predict_morpheme('시5발')
mean_word_predict_morpheme('씨5발')
mean_word_predict_morpheme('시6발')
mean_word_predict_morpheme('씨6발')


99.91% 확률로 욕설입니다.

50.84% 확률로 욕설입니다.

62.81% 확률로 욕설입니다.

64.86% 확률로 욕설이 아닙니다.

52.21% 확률로 욕설이 아닙니다.

65.54% 확률로 욕설이 아닙니다.

53.24% 확률로 욕설이 아닙니다.

62.47% 확률로 욕설이 아닙니다.

50.22% 확률로 욕설입니다.

60.99% 확률로 욕설이 아닙니다.

51.46% 확률로 욕설입니다.



In [ ]:
mean_word_predict_morpheme('ㅅㅐㄲㅣ')
mean_word_predict_morpheme('ㅅㅐ끼')
mean_word_predict_morpheme('ㄱㅐㅅㅐㄲㅣ')

86.95% 확률로 욕설이 아닙니다.

82.85% 확률로 욕설이 아닙니다.

86.95% 확률로 욕설이 아닙니다.



In [ ]:
mean_word_predict_morpheme('시발')
mean_word_predict_morpheme('시빨')
mean_word_predict_morpheme('시팔')
mean_word_predict_morpheme('시빨')
mean_word_predict_morpheme('씨빨')
mean_word_predict_morpheme('ㅅ발')
mean_word_predict_morpheme('ㅅㅣ발')
mean_word_predict_morpheme('ㅆㅣ발')
mean_word_predict_morpheme('ㅆ1발')
mean_word_predict_morpheme('ㅅㅅ1발')
mean_word_predict_morpheme('^^1발')
mean_word_predict_morpheme('^^1발련')
mean_word_predict_morpheme('^^1발놈')
mean_word_predict_morpheme('人人ㅣ발')
mean_word_predict_morpheme('씹알')
mean_word_predict_morpheme('ㅅ시발')
mean_word_predict_morpheme('씨ㅂ알')
mean_word_predict_morpheme('시부랄')
mean_word_predict_morpheme('시부럴')
mean_word_predict_morpheme('시불탱')
mean_word_predict_morpheme('슈발')
mean_word_predict_morpheme('쉬발')
mean_word_predict_morpheme('쉬빨')
mean_word_predict_morpheme('색히')
mean_word_predict_morpheme('색기')
mean_word_predict_morpheme('세끼')
mean_word_predict_morpheme('새퀴')
mean_word_predict_morpheme('쉨')
mean_word_predict_morpheme('샠')
mean_word_predict_morpheme('ㅅㅐ끼')
mean_word_predict_morpheme('ㅅㅐㄲㅣ')


99.91% 확률로 욕설입니다.

60.37% 확률로 욕설입니다.

55.36% 확률로 욕설이 아닙니다.

60.37% 확률로 욕설입니다.

77.54% 확률로 욕설입니다.

84.58% 확률로 욕설입니다.

57.59% 확률로 욕설이 아닙니다.

67.89% 확률로 욕설이 아닙니다.

90.76% 확률로 욕설입니다.

85.31% 확률로 욕설입니다.

62.13% 확률로 욕설이 아닙니다.

92.52% 확률로 욕설입니다.

89.50% 확률로 욕설이 아닙니다.

76.02% 확률로 욕설이 아닙니다.

98.30% 확률로 욕설입니다.

99.94% 확률로 욕설입니다.

56.80% 확률로 욕설입니다.

87.19% 확률로 욕설이 아닙니다.

60.66% 확률로 욕설이 아닙니다.

66.15% 확률로 욕설이 아닙니다.

85.68% 확률로 욕설입니다.

64.67% 확률로 욕설이 아닙니다.

86.95% 확률로 욕설이 아닙니다.

89.84% 확률로 욕설이 아닙니다.

52.94% 확률로 욕설이 아닙니다.

87.31% 확률로 욕설이 아닙니다.

86.95% 확률로 욕설이 아닙니다.

86.70% 확률로 욕설이 아닙니다.

50.55% 확률로 욕설이 아닙니다.

82.85% 확률로 욕설이 아닙니다.

86.95% 확률로 욕설이 아닙니다.

